In [1]:
import numpy as np
import pandas as pd
import pyodbc as pyo
import sqlalchemy as sa
import math
connection_uri = (
    "mssql+pyodbc://TEDPHO0134\TANDT_DP/DP_Tan?driver=ODBC+Driver+17+for+SQL+Server"
)
conn = sa.create_engine(connection_uri)

<h3> Phụ lục: Lấy bảng Period <h3>

In [ ]:
period = pd.DataFrame(columns=['Period', ])

<h4>1. Lấy bảng Commision <h4>

In [7]:
cutoff = '202401'
commission_query = f"select * from AGENT_COM_CUTOFF where CUTOFF like '{cutoff}%'"
agent_com = pd.read_sql(commission_query, conn)

<h4>2. Lấy bảng PY2 <h4>

In [9]:
py2_query = f"select * from AGENT_COM_CUTOFF where CUTOFF like '{cutoff}%'"
py2 = pd.read_sql(py2_query, conn)

<h4>3. Copy bảng <h4>

In [10]:
com_table = agent_com
py2_table = py2

<h4>4. Bổ sung cột liên quan đến CUTOFF <h4>

In [145]:
#Thêm cột ngày xét
Monthly_Com = com_table
Monthly_Com['Date_On_Set'] = "" #Tạo ra cột trống
Monthly_Com['Date_On_Set']= Monthly_Com['Date_On_Set'].str.cat(Monthly_Com['CUTOFF'].str[:4] ,sep ="")
Monthly_Com['Date_On_Set']= Monthly_Com['Date_On_Set'].str.cat(Monthly_Com['CUTOFF'].str[4:6] ,sep ="-")
Monthly_Com['Date_On_Set']= Monthly_Com['Date_On_Set'].str.cat(Monthly_Com['CUTOFF'].str[6:] ,sep ="-")

In [146]:
#Thêm cột ưu tiên
Monthly_Com['Priority'] = Monthly_Com['CUTOFF'].rank(method='dense', ascending=True)

<h4>5. Biến cột Applied Date và Date_On_Set thành kiểu date <h4>

In [148]:
Monthly_Com['Date_On_Set'] = pd.to_datetime(Monthly_Com['Date_On_Set'])
Monthly_Com['Start_Month'] = pd.to_datetime(Monthly_Com['Start_Month'])
Monthly_Com['Applied Premium Date'] = pd.to_datetime(Monthly_Com['Applied Premium Date'])

In [149]:
#Thêm cột ngày đầu tháng:
#-> Đưa vào ngày 1 của tháng
if not('20240229' in Monthly_Com['CUTOFF'].values): 
    Monthly_Com['Start_Month'] = (Monthly_Com['Date_On_Set'].dt.floor('d') + pd.offsets.MonthEnd(0) - pd.offsets.MonthBegin(1))
else:
    Monthly_Com.loc[:,'Start_Month'] = '2024-01-27'

<h4>6. Kỳ I <h4>

In [206]:
Com_I = (Monthly_Com['Applied Premium Date'] >= Monthly_Com['Start_Month']) & \
(Monthly_Com['Applied Premium Date'] <= Monthly_Com['Date_On_Set']) & \
(Monthly_Com['Priority'] == 1) & ( (Monthly_Com['Policy Status'] != 'SU') | (Monthly_Com['Policy Status'] != 'TR'))
Monthly_Com_I = Monthly_Com.loc[Com_I]

In [207]:
#Lấy ra các HD trong kỳ I
com_pe_1 = Monthly_Com_I

In [221]:
com_pe_1['Receive Policy date'].to_string

<bound method Series.to_string of 175       2016-04-11
198       2016-04-11
199       2016-04-11
200       2016-04-11
201       2016-04-11
             ...    
1565994   2020-01-21
1565995   2020-01-21
1565996   2020-01-21
1566197   2020-02-28
1573023   2020-03-26
Name: Receive Policy date, Length: 6115, dtype: datetime64[ns]>

In [ ]:
com_pe_1.loc[(com_pe_1['Freelook'] <= 21), 'Status'] = "1" #Chưa pass FL
com_pe_1.loc[(com_pe_1['Receive Policy date'].str.len() <1), 'Status'] = "2" #Chưa có ACK
com_pe_1.loc[(com_pe_1['Mark_Status'].str.len() > 1) & (com_pe_1['Terminated Date'].str.len() <1), 'Status'] = "3" #FL trong 21 ngày
com_pe_1.loc[(com_pe_1['Mark_Status'].str.len() < 1) & (com_pe_1['Terminated Date'].str.len() >1), 'Status'] = "4" #Ter trước khi phát sinh phí
com_pe_1.loc[(com_pe_1['Mark_Status'].str.len() < 1) & (com_pe_1['Freelook'] > 21) , 'Status'] = "5"

In [229]:
import numpy as np
com_pe_1['Status'] = np.where(com_pe_1['Freelook']<=21, "1", 
                     np.where(com_pe_1['Receive Policy date'].isnull(), "2",
                     np.where((com_pe_1['Policy Status'] == "FL") & (com_pe_1['Terminated Date'].isnull()), "3",
                     np.where((com_pe_1['Policy Status'] != "FL") & (com_pe_1['Terminated Date'].notnull()), "4", "5")     
                             )))

C:\Users\vagy1255\AppData\Local\Temp\ipykernel_21648\1145856748.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  com_pe_1['Status'] = np.where(com_pe_1['Freelook']<=21, "1",


In [166]:
#Lấy ra các HD 

In [230]:
com_pe_1.to_csv("check.csv")